In [40]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq

# setup the tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def square(a: int) -> int:
    """Calculates the square of a number."""
    a = int(a)
    return a * a

@tool
def give_increment(a: int) -> float:
    """if the number is negative then make it postive number and then return the number * 1.25"""
    result = abs(a) * 1.25
    return result

@tool
def give_customise_increment(a: int, b: int) -> float:
    """if the number is negative then make it postive number and return the number * b/100"""
    result = abs(a) * b/100
    return result

@tool
def add_salution_to_name(name: str) -> str:
    """Modify then name with which starts with capital letter and prefix a salution before the name such as Mr. or Mrs.
    According to gender of the name, it will add Mr. or Mrs. before the name.
    If you unable to identify the name then return the name with Prefix 'Hello Dear'
    """
    # return "Mr. " + name

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a mathematical assistant.
        Use your tools to answer questions. If you do not have a tool to
        answer the question, say so. 

        Return answer with description in a new line, e.g
        Human: What is 1 + 1?
        AI: 1 + 1 = 2
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Choose the LLM that will drive the agent
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

# setup the toolkit
toolkit = [add, multiply, square, give_increment, give_customise_increment, add_salution_to_name]

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

print(agent_executor)


verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMess

In [41]:
print("-----------------")
result = agent_executor.invoke({"input": "what is 1 + 1?"})
print(result['output'])
print("+++++++++++++++++")

print("-----------------")
result = agent_executor.invoke({"input": "how much to give on increment on -300 and 150"})
print(result['output'])
print("+++++++++++++++++")

print("-----------------")
result = agent_executor.invoke({"input": "how much to give on increment on 1000 with 10%"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 1}`


22

> Finished chain.
2
+++++++++++++++++
-----------------


> Entering new AgentExecutor chain...

Invoking: `give_increment` with `{'a': -300}`


375.0
Invoking: `give_increment` with `{'a': 150}`


187.5The increment on -300 is 375.0, and the increment on 150 is 187.5. So, the final answer is:

The increment on -300 is 375.0, and the increment on 150 is 187.5.

Note: The increment on -300 is 375.0 because the tool "give_increment" makes a negative number positive and then multiplies it by 1.25.

> Finished chain.
The increment on -300 is 375.0, and the increment on 150 is 187.5. So, the final answer is:

The increment on -300 is 375.0, and the increment on 150 is 187.5.

Note: The increment on -300 is 375.0 because the tool "give_increment" makes a negative number positive and then multiplies it by 1.25.
+++++++++++++++++
-----------------


> Entering new AgentExecutor chain...

In

In [42]:
print("-----------------")
result = agent_executor.invoke({"input": "my name is krishna bhunia and I need to address me properly"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...
Mr. Krishna Bhunia!

> Finished chain.
Mr. Krishna Bhunia!
+++++++++++++++++


In [43]:
print("-----------------")
result = agent_executor.invoke({"input": "My friend name is priyanka and abhijit"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add_salution_to_name` with `{'name': 'Priyanka'}`


None
Invoking: `add_salution_to_name` with `{'name': 'Abhijit'}`


NoneMr. Abhijit

> Finished chain.
Mr. Abhijit
+++++++++++++++++


In [44]:
print("-----------------")
result = agent_executor.invoke({"input": "My friend name is Alex"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add_salution_to_name` with `{'name': 'Alex'}`


None
Invoking: `add_salution_to_name` with `{'name': 'Alex'}`
responded: I apologize for the inconvenience. It seems that the tool call "call_92xn" did not yield a valid result. Let me try again.



NoneHello Dear Alex

> Finished chain.
Hello Dear Alex
+++++++++++++++++
